In [68]:
# processing the dictionary objects

In [69]:
# altair to visualize the data. read in pickle
# determine df shape for this to work
# import birdseye and snoop
import snoop
import altair as alt
# from vega_datasets import data
from typing import Optional, Dict, List, Any, Union, Tuple
import pydantic
from pydantic import ValidationError, validator, root_validator, Field, constr, BaseModel
from pydantic_docx import Docx_Paragraph_and_Runs, read_docx, extract_features #type:ignore
from pydantic_docx_processor import create_sized_dataframe, expand_dataframe #type:ignore
from pydantic_fula_dictionary_entries_v2 import dict_entry, pydantic_lemma, pydantic_root, pydantic_root_subpiece, parse_dataframe_groups #type:ignore
import re
import json
from itertools import compress, chain
from datetime import datetime
import pandas as pd
import numpy as np
from functools import partial
import pickle
import openpyxl

In [70]:
b = [True, True, False]
s = ['a','b','c']
text = ''.join(chain(compress(s,b))).strip()
print(text)
print(list(compress(s,b)))

ab
['a', 'b']


## Loading Data

In [71]:
# from collections import Counter

In [72]:
with open('pickled_results/all_dictionary_entries_list_test.pkl', 'rb') as file:
   all_entries = pickle.load(file, encoding='utf-8')

In [73]:
print(all_entries[-1])

subsidiary_paragraphs=[Docx_Paragraph_and_Runs(para_first_line_indent=-91440, paragraph_enumeration=32505, run_text=['to give (push with?) the head to get out ?; to inflate'], para_text='to give (push with?) the head to get out ?; to inflate', run_font_size_pt=[None], run_bold=[None], para_left_indent=70239053070000, run_italic=[None], run_font_name=['TmsRmn 10pt']), Docx_Paragraph_and_Runs(para_first_line_indent=-91440, paragraph_enumeration=32506, run_text=['donner (de?) la tête pour sortir; gonfler'], para_text='donner (de?) la tête pour sortir; gonfler', run_font_size_pt=[None], run_bold=[None], para_left_indent=70239053070000, run_italic=[None], run_font_name=['TmsRmn 10pt'])] irregularities=[] root=pydantic_root(index=32503, paragraph=Docx_Paragraph_and_Runs(para_first_line_indent=-182880, paragraph_enumeration=32503, run_text=['ƳUUƳ-  (2?)'], para_text='ƳUUƳ-  (2?)', run_font_size_pt=[12.0], run_bold=[None], para_left_indent=46826035380000, run_italic=[None], run_font_name=[None

### Checking data contents

In [74]:
other_list = []
missing_pos = []
count = 0
for e in all_entries:

   plurals = re.findall(r"([^\/]+)",e.lemma.paragraph.para_text)
   if len(plurals)==2:
      count+=1
      # print(e.lemma.paragraph.para_text)
      # print(plurals)
   elif len(plurals)>2:
      print(e.lemma.paragraph.para_text)
      #search helv, slice runs list
      [print(i,'\t',p) for i,p in enumerate(plurals)]
      # print(e.lemma.paragraph)
print(count)

alaada (o) / alaadaaji (ɗi)  DFzbu  Z<G,A>  [aada/aadaaji]:D,Z<FT> [laada/laadaaji]:DF  [aadi]:Z  [sunna, tawaangal]
0 	 alaada (o) 
1 	  alaadaaji (ɗi)  DFzbu  Z<G,A>  [aada
2 	 aadaaji]:D,Z<FT> [laada
3 	 laadaaji]:DF  [aadi]:Z  [sunna, tawaangal]
Aaraabo, Aarabo (o) / Aaraɓɓe, Aaraɓe'en (ɓe)  DFcz  CZ<G,A>  [Araabo/Araaɓe]:W  [Ara(a)buujo, Laarabuujo]
0 	 Aaraabo, Aarabo (o) 
1 	  Aaraɓɓe, Aaraɓe'en (ɓe)  DFcz  CZ<G,A>  [Araabo
2 	 Araaɓe]:W  [Ara(a)buujo, Laarabuujo]
aawdi (ndi) / aawɗi (ɗi)  FCz  C<->,Z<A,G>  [awdi/awdiiji]
0 	 aawdi (ndi) 
1 	  aawɗi (ɗi)  FCz  C<->,Z<A,G>  [awdi
2 	 awdiiji]
atiime, atiimeejo (o) / atiimeeɓe, atiime'en (ɓe)  DFCZH  Z<M,BF>  [atimeejo/atimeeɓe, atime'en]  [yaatime, yatiimaajo]
0 	 atiime, atiimeejo (o) 
1 	  atiimeeɓe, atiime'en (ɓe)  DFCZH  Z<M,BF>  [atimeejo
2 	 atimeeɓe, atime'en]  [yaatime, yatiimaajo]
bammbaaɗo (o) / wammbayɓe, wammbaaɓe (ɓe)  DFZEHi  Z<->  [bammbaajo/bammbaaɓe]:I
0 	 bammbaaɗo (o) 
1 	  wammbayɓe, wammbaaɓe (ɓe)  DFZEHi  Z<

In [75]:
other_list = []
missing_pos = []
count = 0
for e in all_entries:
   if e.lemma.FulaPOSClass == ['Other_']:
      # print(e.lemma.paragraph.run_text)
      other_list.append(e)
   elif e.lemma.FulaPOSClass == ['None']:
      missing_pos.append(e)

      

In [84]:
l = []
pos_config = {'docxFeature': 'run_italic',
         'strSummary':'fontItalic', 
         'value':True}

      #POS
for i,e in enumerate(other_list):
   (is_target, text, target_mask, run_text) = extract_features(e.lemma.paragraph,pos_config)
   assert not isinstance(run_text,bool)
   assert not isinstance(target_mask,bool)
   matches = ','.join(chain(compress(run_text,target_mask)))
   print(e.lemma.paragraph)
   if len(matches)>0:
      l.append((matches))
# other_list[1].lemma.paragraph
# print(l)
from collections import Counter
italic_words = Counter()
matches = []
for m in l:
   # print(m)
   # print(m.strip().replace(' ',',').split(','))
   matches.extend(m.strip().replace(' ',',').split(','))

sorted(Counter(matches).items(),key=lambda x: (-x[1],x[0]))

para_first_line_indent=-91440 paragraph_enumeration=5 run_text=['a  ', 'prn,sbj,sf  ', 'DFZH  Z<->'] para_text='a  prn,sbj,sf  DFZH  Z<->' run_font_size_pt=[None, None, 8.0] run_bold=[True, None, None] para_left_indent=46826035380000 run_italic=[None, True, None] run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt']
para_first_line_indent=-91440 paragraph_enumeration=8 run_text=['-a  ', 'suf,pos  ', 'F'] para_text='-a  suf,pos  F' run_font_size_pt=[None, None, 8.0] run_bold=[True, None, None] para_left_indent=46826035380000 run_italic=[None, True, None] run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt']
para_first_line_indent=-91440 paragraph_enumeration=11 run_text=['aan  ', 'prn,ind  ', 'DFZH  [an]:Z<->'] para_text='aan  prn,ind  DFZH  [an]:Z<->' run_font_size_pt=[None, None, 8.0] run_bold=[True, None, None] para_left_indent=46826035380000 run_italic=[None, True, None] run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt']
para_first_line_indent=-91440 paragraph_enumeratio

[('prn', 84),
 ('PN', 62),
 ('pos', 35),
 ('adj.', 32),
 ('/', 28),
 ('ind', 26),
 ('c.i.', 24),
 ('sbj', 15),
 ('interj.', 12),
 ('ideoph.', 11),
 ('part.', 11),
 ('interj', 9),
 ('conj.', 8),
 ('interrog.', 8),
 ('lf', 8),
 ('obj', 8),
 ('suf', 7),
 ('prep.', 6),
 ('sf', 6),
 ('excl.', 5),
 ('', 4),
 ('indic.', 4),
 ('interrog', 4),
 ('relat.', 2),
 ('?', 1),
 ('adj', 1),
 ('alalba', 1),
 ('alarba', 1),
 ('alkamisa', 1),
 ('altine', 1),
 ('alwaaji', 1),
 ('battaneeji', 1),
 ('bunnaaji', 1),
 ('conj&prep', 1),
 ('conj.?', 1),
 ('daŋeeji', 1),
 ('debolobbooje', 1),
 ('dibonaaji', 1),
 ('fusunaaji', 1),
 ('fusunnaaji', 1),
 ('geewti', 1),
 ('gori', 1),
 ('ideoph.?', 1),
 ('iimaji', 1),
 ('indic', 1),
 ('jale', 1),
 ('kelɗe', 1),
 ('kiitaaji', 1),
 ('kiɓe', 1),
 ('koreeji', 1),
 ('kummbeeji', 1),
 ("naa'ibuuji", 1),
 ('ngeewtaaji', 1),
 ('peruuji', 1),
 ('salluuji', 1),
 ('sari', 1),
 ('sarraaji', 1),
 ('siidooji', 1),
 ('sorlaaji', 1),
 ('talata', 1),
 ('ɓadaaji', 1),
 ('ɓadeeji', 1)]

In [110]:
run_text = ['bunna  ', ' /  bunnaaji  ', 'DZ  Z<->  [fanna, fuuna]']
target_mask = [False,True,False]
matches = list(compress(run_text,target_mask))
text = ';'.join(matches) #TODO this only pulls POS info from the first contiguous italics run
# if len(matches)>1:
   # print(self.paragraph.run_text)
word_class = re.findall(r"\(([^\)]+)\)",text) #anything between parenthesis
is_verb = bool(re.findall(r"v.",text.lower()))
noun_flag = re.findall(r"(n\.)",text) #find n.        
plurals = re.findall(r"([^\/]+)",text) #anything not a /
# plurals = text.index("/") #anything not a /
print(re.search(r"(/)",text).start())
plurals = text[re.search(r"(/)",text).end():] #anything not a /
print(plurals)
if any([len(word_class)>0, noun_flag, len(plurals)>1]):
   print('here')

1
  bunnaaji  
here


In [47]:
ct = 0
# entry
temp_list = []
for e in all_entries:
   para = e.root.paragraph
   temp_list.append({
      'entity': 'root',
      'index': e.root.index,
      'run_font_size_pt': para.run_font_size_pt,
      'font_name': para.run_font_name,
      'bold': para.run_bold,
      'italic': para.run_italic
      })
   try:
      para = e.root_subpiece.paragraph
      temp_list.append({
         'entity': 'root_subpiece',
         'index': e.root_subpiece.index,
         'run_font_size_pt': para.run_font_size_pt,
         'font_name': para.run_font_name,
         'bold': para.run_bold,
         'italic': para.run_italic
         })
   except:
      pass
   para = e.lemma.paragraph
   temp_list.append({
      'entity': 'lemma',
      'index': e.lemma.index,
      'run_font_size_pt': para.run_font_size_pt,
      'font_name': para.run_font_name,
      'bold': para.run_bold,
      'italic': para.run_italic
      })
   for i,p in enumerate(e.subsidiary_paragraphs):
      temp_list.append({
      'entity': 'subsidiary_paragraphs',
      'index': 1+i+e.lemma.index,
      'run_font_size_pt': p.run_font_size_pt,
      'font_name': p.run_font_name,
      'bold': p.run_bold,
      'italic': p.run_italic
      })

   # print(e.lemma.dict())
   # break
#    if len(e.irregularities) > 0:
#       print(e.irregularities)
#       ct+=1
#       print(e.lemmaLine_runs)
# print(ct)

In [48]:
df = pd.DataFrame(temp_list).drop_duplicates(subset='index')
# print(df.shape)
# print(df.drop_duplicates(subset='index').shape)
df = df.explode(['run_font_size_pt','font_name','bold','italic'])
for name, tdf in df.groupby(['entity','run_font_size_pt','font_name','bold','italic']).groups.items():
   print(name,tdf.shape)
   if name == ('subsidiary_paragraphs', 8.0, 'Helv 8pt', np.nan, np.nan):
      print(tdf)

('lemma', nan, 'TmsRmn 10pt', True, nan) (8026,)
('lemma', nan, 'TmsRmn 10pt', nan, True) (7908,)
('lemma', 8.0, 'Helv 8pt', nan, nan) (7984,)
('root', 8.0, 'Helv 8pt', nan, nan) (533,)
('root', 12.0, nan, nan, nan) (4939,)
('root_subpiece', 12.0, nan, nan, nan) (693,)
('subsidiary_paragraphs', nan, 'TmsRmn 10pt', nan, nan) (16397,)
('subsidiary_paragraphs', 8.0, 'TmsRmn 8pt', nan, nan) (837,)
('subsidiary_paragraphs', nan, 'TmsRmn 10pt', nan, True) (454,)
('subsidiary_paragraphs', nan, 'TmsRmn 10pt', True, nan) (40,)
('subsidiary_paragraphs', 8.0, 'Helv 8pt', nan, nan) (2,)
Int64Index([14313, 14314], dtype='int64')


In [49]:
temp_list[14313] #13925
temp_list[14314] #13926
for e in all_entries:
   if e.lemma.index == 13925-1:
      print(e)

subsidiary_paragraphs=[Docx_Paragraph_and_Runs(para_first_line_indent=-91440, paragraph_enumeration=13925, run_text=['coffee'], para_text='coffee', run_font_size_pt=[8.0], run_bold=[None], para_left_indent=70239053070000, run_italic=[None], run_font_name=['Helv 8pt']), Docx_Paragraph_and_Runs(para_first_line_indent=-91440, paragraph_enumeration=13926, run_text=['café'], para_text='café', run_font_size_pt=[8.0], run_bold=[None], para_left_indent=70239053070000, run_italic=[None], run_font_name=['Helv 8pt'])] irregularities=[] root=pydantic_root(index=13923, paragraph=Docx_Paragraph_and_Runs(para_first_line_indent=-182880, paragraph_enumeration=13923, run_text=['KAFE    ', 'Fr'], para_text='KAFE    Fr', run_font_size_pt=[12.0, 8.0], run_bold=[None, None], para_left_indent=46826035380000, run_italic=[None, None], run_font_name=[None, 'Helv 8pt']), text='KAFE', line_runs=['KAFE    ', 'Fr'], origin=None, notes=None) root_subpiece=None lemma=pydantic_lemma(index=13924, paragraph=Docx_Paragra

In [50]:
for name in df.reset_index().groupby(['font_name','bold','italic']).groups:
   print(name)

('Helv 8pt', nan, nan)
('TmsRmn 10pt', True, nan)
('TmsRmn 10pt', nan, True)
('TmsRmn 10pt', nan, nan)
('TmsRmn 8pt', nan, nan)
(nan, nan, nan)


In [51]:
dialects = []
typ_df = pd.DataFrame()
feat_df = pd.DataFrame()
for i,e in enumerate(all_entries):
   typ_dct = {}
   for k,v in e.dict().items():
      typ_dct[k]=type(v)
   tdf = pd.Series(typ_dct).to_frame().T
   fdf = pd.Series(e.dict()).to_frame().T
   typ_df = pd.concat([typ_df,tdf],ignore_index=True)
   feat_df = pd.concat([feat_df,fdf],ignore_index=True)


In [52]:
rare_cols = typ_df.describe().T
rare_cols = rare_cols[rare_cols['unique']>1]
rare_cols = rare_cols[rare_cols['top'].isin([type(True)])].T
# rare_cols

unused_cols = typ_df.describe().T
unused_cols = unused_cols[unused_cols['unique']==1]
unused_cols = unused_cols[unused_cols['top'].isin([type(True)])]
unused_cols = unused_cols.T
unused_cols

""
count
unique
top
freq


In [57]:
lemmas_df = pd.DataFrame.from_dict(df.lemma.to_list())
lemmas_df

,index,paragraph,text,line_runs,irregularities,unusedContent,FulaDialects,FulaPOSTags,FulaPOSClass,FulaNoun_Classes,FulaNoun_Plurals,FulaSynonyms,FulaCrossRef
0,5,"{'para_first_line_indent': -91440, 'paragraph_...",a,"[a , prn,sbj,sf , DFZH Z<->]",None,None,{-},"[prn, sbj, sf ]",[Other_],None,None,None,None
1,8,"{'para_first_line_indent': -91440, 'paragraph_...",-a,"[-a , suf,pos , F]",None,None,{Mali},"[suf, pos ]",[Other_],None,None,None,None
2,11,"{'para_first_line_indent': -91440, 'paragraph_...",aan,"[aan , prn,ind , DFZH [an]:Z<->]",None,None,{-},"[prn, ind ]",[Other_],None,None,None,None
3,14,"{'para_first_line_indent': -91440, 'paragraph_...",aɗa,"[aɗa , prn,sbj,lf , DFZH [aɗo]:Z]",None,None,{Mali},"[prn, sbj, lf ]",[Other_],None,None,None,None
4,17,"{'para_first_line_indent': -91440, 'paragraph_...","-ɗaa, -ɗa","[-ɗaa, -ɗa , prn,sbj , DF]",None,None,{Mali},"[prn, sbj ]",[Other_],None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7980,32488,"{'para_first_line_indent': -91440, 'paragraph_...",ƴuugo,"[ƴuugo , (ngo) / ƴuuge (ɗe) , Z [naɗɗire]]",None,None,{Mali},[(ngo) / ƴuuge (ɗe) ],[Noun],"[ngo, ɗe]","[(ngo) , ƴuuge (ɗe) ]",None,None
7981,32492,"{'para_first_line_indent': -91440, 'paragraph_...",ƴuulde,"[ƴuulde , (nde) , D]",None,None,{Mali},[(nde) ],[Noun],[nde],[(nde) ],None,None
7982,32496,"{'para_first_line_indent': -91440, 'paragraph_...",ƴuuwaade,"[ƴuuwaade , v.mv , D [ƴuubaade, ƴooraade]]",None,None,{Mali},[v.mv ],[Verb],None,None,None,None
7983,32500,"{'para_first_line_indent': -91440, 'paragraph_...",ƴuuƴude,"[ƴuuƴude , v.av , D]",None,None,{Mali},[v.av ],[Verb],None,None,None,None


In [59]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 30)
# df = feat_df[feat_df.columns.difference(unused_cols.columns)]
# df = df.set_index('lemma_index')
lemmas_df.explode('FulaDialects')

,index,paragraph,text,line_runs,irregularities,unusedContent,FulaDialects,FulaPOSTags,FulaPOSClass,FulaNoun_Classes,FulaNoun_Plurals,FulaSynonyms,FulaCrossRef
0,5,"{'para_first_line_indent': -91440, 'paragraph_...",a,"[a , prn,sbj,sf , DFZH Z<->]",None,None,-,"[prn, sbj, sf ]",[Other_],None,None,None,None
1,8,"{'para_first_line_indent': -91440, 'paragraph_...",-a,"[-a , suf,pos , F]",None,None,Mali,"[suf, pos ]",[Other_],None,None,None,None
2,11,"{'para_first_line_indent': -91440, 'paragraph_...",aan,"[aan , prn,ind , DFZH [an]:Z<->]",None,None,-,"[prn, ind ]",[Other_],None,None,None,None
3,14,"{'para_first_line_indent': -91440, 'paragraph_...",aɗa,"[aɗa , prn,sbj,lf , DFZH [aɗo]:Z]",None,None,Mali,"[prn, sbj, lf ]",[Other_],None,None,None,None
4,17,"{'para_first_line_indent': -91440, 'paragraph_...","-ɗaa, -ɗa","[-ɗaa, -ɗa , prn,sbj , DF]",None,None,Mali,"[prn, sbj ]",[Other_],None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7980,32488,"{'para_first_line_indent': -91440, 'paragraph_...",ƴuugo,"[ƴuugo , (ngo) / ƴuuge (ɗe) , Z [naɗɗire]]",None,None,Mali,[(ngo) / ƴuuge (ɗe) ],[Noun],"[ngo, ɗe]","[(ngo) , ƴuuge (ɗe) ]",None,None
7981,32492,"{'para_first_line_indent': -91440, 'paragraph_...",ƴuulde,"[ƴuulde , (nde) , D]",None,None,Mali,[(nde) ],[Noun],[nde],[(nde) ],None,None
7982,32496,"{'para_first_line_indent': -91440, 'paragraph_...",ƴuuwaade,"[ƴuuwaade , v.mv , D [ƴuubaade, ƴooraade]]",None,None,Mali,[v.mv ],[Verb],None,None,None,None
7983,32500,"{'para_first_line_indent': -91440, 'paragraph_...",ƴuuƴude,"[ƴuuƴude , v.av , D]",None,None,Mali,[v.av ],[Verb],None,None,None,None


## Exploring Dialects Field

In [60]:

multi_len = lemmas_df['FulaDialects'].apply(lambda x: x if isinstance(x,list) and len(x)>1 else None)
multi_len = multi_len[multi_len.notna()]
# print(df.shape)
# df.explode('FulaDialects')['FulaDialects']
multi_len

Series([], Name: FulaDialects, dtype: object)

In [61]:
temp = lemmas_df['FulaDialects'].apply(lambda x: ','.join(x))
temp = temp.apply(lambda x: x.split(','))
temp

0          [-]
1       [Mali]
2          [-]
3       [Mali]
4       [Mali]
         ...  
7980    [Mali]
7981    [Mali]
7982    [Mali]
7983    [Mali]
7984    [Mali]
Name: FulaDialects, Length: 7985, dtype: object

In [62]:
lists = temp.apply(lambda x: isinstance(x,list))
lens = temp.apply(lambda x: len(x))
# multi_len = multi_len[multi_len.notna()]
# print(df.shape)
print(lists.describe())
print(lens.describe())
# df.explode('FulaDialects')['FulaDialects']
# multi_len.sample(1).values

count     7985
unique       1
top       True
freq      7985
Name: FulaDialects, dtype: object
count    7985.000000
mean        1.488040
std         1.316569
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        13.000000
Name: FulaDialects, dtype: float64


## Processing Dialect field contents

In [63]:
dialected_df = lemmas_df.copy()
dialected_df['FulaDialects'] = dialected_df['FulaDialects'].apply(lambda x: ','.join(x)).apply(lambda x: x.split(','))


### Calculating Counts

In [65]:
dialected_df = dialected_df.explode('FulaDialects')
counts = dialected_df[['FulaDialects','index']]
counts.drop_duplicates()

,FulaDialects,index
0,-,5
1,Mali,8
2,-,11
3,Mali,14
4,Mali,17
...,...,...
7980,Mali,32488
7981,Mali,32492
7982,Mali,32496
7983,Mali,32500


In [66]:
dialected_df[dialected_df['FulaDialects']=='FJ']

,index,paragraph,text,line_runs,irregularities,unusedContent,FulaDialects,FulaPOSTags,FulaPOSClass,FulaNoun_Classes,FulaNoun_Plurals,FulaSynonyms,FulaCrossRef
13,47,"{'para_first_line_indent': -91440, 'paragraph_...",aadamanke(ejo),"[aadamanke(ejo) , (o) / aadamankeeɓe (ɓe) , D...",None,None,FJ,[(o) / aadamankeeɓe (ɓe) ],[Noun],"[o, ɓe]","[(o) , aadamankeeɓe (ɓe) ]",None,None
28,110,"{'para_first_line_indent': -91440, 'paragraph_...",aatude,"[aatude , v.av , DcZH C<FJ>,Z<FJ,M> cf.:[h...",None,None,FJ,[v.av ],[Verb],None,None,None,None
28,110,"{'para_first_line_indent': -91440, 'paragraph_...",aatude,"[aatude , v.av , DcZH C<FJ>,Z<FJ,M> cf.:[h...",None,None,FJ,[v.av ],[Verb],None,None,None,None
37,146,"{'para_first_line_indent': -91440, 'paragraph_...","abajada, abajadda","[abajada, abajadda , (o) / abajadaaji (ɗi) , ...",None,None,FJ,[(o) / abajadaaji (ɗi) ],[Noun],"[o, ɗi]","[(o) , abajadaaji (ɗi) ]",None,None
42,170,"{'para_first_line_indent': -91440, 'paragraph_...",adaade,"[adaade , v.pv , DFZ C<FJ>,Z<-> [artaade]]",None,None,FJ,[v.pv ],[Verb],None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7934,32306,"{'para_first_line_indent': -91440, 'paragraph_...",ƴettude,"[ƴettude , v.av , DCZ CZ<FJ,FT,M,BF> [ƴept...",None,None,FJ,[v.av ],[Verb],None,None,None,None
7959,32408,"{'para_first_line_indent': -91440, 'paragraph_...","ƴoyre, ƴoƴre","[ƴoyre, ƴoƴre , (nde) / ƴoƴe (ɗe) , DFCZ ...",None,None,FJ,[ (nde) / ƴoƴe (ɗe) ],[Noun],"[nde, ɗe]","[ (nde) , ƴoƴe (ɗe) ]",None,None
7968,32444,"{'para_first_line_indent': -91440, 'paragraph_...","ƴumndi, ƴunndi","[ƴumndi, ƴunndi , (ndi) , CZ <FJ,FT,M,BF> [...",None,None,FJ,[(ndi) ],[Noun],[ndi],[(ndi) ],None,None
7978,32482,"{'para_first_line_indent': -91440, 'paragraph_...",ƴuugaaɗo,"[ƴuugaaɗo , (o) / ƴuugaaɓe (ɓe) , cZ C<FJ>,Z...",None,None,FJ,[(o) / ƴuugaaɓe (ɓe) ],[Noun],"[o, ɓe]","[(o) , ƴuugaaɓe (ɓe) ]",None,None


In [ ]:
dialected_df

,FulaAnnotations_runs,FulaDialects,FulaNoun_NounsAndClass,FulaNoun_PluralsAndClass,FulaPOSClass,FulaPOSTags,englishGlossLine_runs,frenchGlossLine_runs,irregularities,lemma,lemmaLine_runs,lemmaLine_unusedContent,lemma_index,paragraphs,root,root_metadata,root_subpiece
0,True,-,False,True,True,"[prn, sbj, sf ]",[you (sg.)],[tu],[],a,"[a , prn,sbj,sf , DFZH Z<->]","[(1, prn,sbj,sf ), (2, DFZH Z<->)]",5,"[{'para_first_line_indent': -91440, 'paragraph...",A,"{'root_index': 4, 'root_runs': ['A']}",False
1,True,Mali,False,True,True,"[suf, pos ]",[your (sg.) (only with certain nouns such as t...,"[ton, ta, tes (seulement pour certains substan...",[],-a,"[-a , suf,pos , F]","[(1, suf,pos ), (2, F)]",8,"[{'para_first_line_indent': -91440, 'paragraph...",A,"{'root_index': 4, 'root_runs': ['A']}",False
2,True,-,False,True,True,"[prn, ind ]",[you (sg.) (emphatic)],[toi (emphatique)],[],aan,"[aan , prn,ind , DFZH [an]:Z<->]","[(1, prn,ind ), (2, DFZH [an]:Z<->)]",11,"[{'para_first_line_indent': -91440, 'paragraph...",A,"{'root_index': 4, 'root_runs': ['A']}",False
3,True,Mali,False,True,True,"[prn, sbj, lf ]",[you (sg.)],[tu],[],aɗa,"[aɗa , prn,sbj,lf , DFZH [aɗo]:Z]","[(1, prn,sbj,lf ), (2, DFZH [aɗo]:Z)]",14,"[{'para_first_line_indent': -91440, 'paragraph...",A,"{'root_index': 4, 'root_runs': ['A']}",False
4,True,Mali,False,True,True,"[prn, sbj ]",[you (sg.) (used with imperfect and subjunctiv...,[tu (employé avec les formes imparfaites et su...,[],"-ɗaa, -ɗa","[-ɗaa, -ɗa , prn,sbj , DF]","[(1, prn,sbj ), (2, DF)]",17,"[{'para_first_line_indent': -91440, 'paragraph...",A,"{'root_index': 4, 'root_runs': ['A']}",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7980,True,Mali,"[(ƴuugo, ngo)]","[(ngo) , ƴuuge (ɗe) ]",[Noun],True,[hump],[bosse],[],ƴuugo,"[ƴuugo , (ngo) / ƴuuge (ɗe) , Z [naɗɗire]]","[(1, (ngo) / ƴuuge (ɗe) ), (2, Z [naɗɗire])]",32488,"[{'para_first_line_indent': -91440, 'paragraph...",ƳUUG-,"{'root_index': 32481, 'root_runs': ['ƳUUG-']}",False
7981,True,Mali,"[(ƴuulde, nde)]",True,[Noun],True,"[boil, furuncle]",[furoncle],[],ƴuulde,"[ƴuulde , (nde) , D]","[(1, (nde) ), (2, D)]",32492,"[{'para_first_line_indent': -91440, 'paragraph...",ƳUULDE,"{'root_index': 32491, 'root_runs': ['ƳUULDE']}",False
7982,True,Mali,False,True,[Verb],[v.mv ],[to incline; to rear up],[s'incliner; se cabrer],[],ƴuuwaade,"[ƴuuwaade , v.mv , D [ƴuubaade, ƴooraade]]","[(1, v.mv ), (2, D [ƴuubaade, ƴooraade])]",32496,"[{'para_first_line_indent': -91440, 'paragraph...",ƳUUW-,"{'root_index': 32495, 'root_runs': ['ƳUUW-']}",False
7983,True,Mali,False,True,[Verb],[v.av ],[to increase in volume],[augmenter de volume],[],ƴuuƴude,"[ƴuuƴude , v.av , D]","[(1, v.av ), (2, D)]",32500,"[{'para_first_line_indent': -91440, 'paragraph...",ƳUUƳ-,"{'root_index': 32499, 'root_runs': ['ƳUUƳ-']}",False


In [67]:
dialects_descriptions = pd.read_excel('test_data/abbr_pdf_text.xlsx')
dialects_descriptions = dialects_descriptions[['FulaDialects','pg_summarized_pdf_text','pdf_text']]

counts = dialected_df[['FulaDialects','index']]
counts = counts.drop_duplicates().groupby('FulaDialects').count()
# print(counts)
counts.columns = ['lemma_counts']
counts.to_csv('lemma_counts_by_dialect.csv',encoding='utf-8-sig')


counts = counts.merge(dialects_descriptions,how='inner',on='FulaDialects')


dialects_descriptions = counts.to_excel('test_data/abbr_pdf_text.xlsx',encoding='utf-8-sig')
counts

,FulaDialects,lemma_counts,pg_summarized_pdf_text,pdf_text
0,+,197,multiple forms have multiple cited dialects in...,"Indicates that the source (e.g. C ( +) , Z( +}..."
1,-,1557,no dialect in source citatiob,Indicates that the source whose abbreviation a...
2,?,1,NaN,NaN
3,A,366,Adamawa,Adamawa
4,A(Jam'aare),2,Cameroonian Adamawa,(zone foulaphone du Cameroun et sous-dialecte ...
...,...,...,...,...
22,NE,191,eastern Niger,"Niger oriental, eastern Niger"
23,NO,163,western Niger,"Niger occidental, western Niger"
24,S,5,Sokoto,Sokoto
25,SH,24,"Seeno-Hayre, east of Masina",Seeno-Hayre (zone foulaphone qui se trouve a I...


### Exploring low-count dialects

In [ ]:
# dialected_dct = {}
small_dialects = {}

# counts = dialected_df[['FulaDialects','lemma_index']]
# for k,v in counts.drop_duplicates().groupby('FulaDialects'):
for k,v in pd.Series(data = counts['lemma_counts'].values, index = counts['FulaDialects']).items():
   # print(type(v))
   # dialected_dct[k]=v
   # print(k,'\t',v)
   if v < 10:
      small_dialects[k]=v 
      print(k,'\t',v)


? 	 1
A(Jam'aare) 	 2
F 	 2
FN 	 8
Gimmballa 	 1
Gimmballa(Ñafunke) 	 2
K 	 3
Keñeba 	 4
Motti 	 1
S 	 5
Soosooɓe(Guumel) 	 1


In [ ]:
sd = list(small_dialects.keys())
print(small_dialects)
# print(sd)
list(small_dialects[sd[9]].index)

{'?':     FulaDialects lemma_index
191            ?         815, "A(Jam'aare)":      FulaDialects lemma_index
49    A(Jam'aare)         195
5473  A(Jam'aare)       22792, 'F':      FulaDialects lemma_index
5296            F       21954
5581            F       23189, 'FN':      FulaDialects lemma_index
1122           FN        4848
1308           FN        5612
1940           FN        8092
4618           FN       19123
4743           FN       19649
6852           FN       28122
7388           FN       30216
7852           FN       31975, 'Gimmballa':     FulaDialects lemma_index
307    Gimmballa        1332, 'Gimmballa(Ñafunke)':             FulaDialects lemma_index
4226  Gimmballa(Ñafunke)       17488
4250  Gimmballa(Ñafunke)       17574, 'K':      FulaDialects lemma_index
1817            K        7597
1845            K        7707
2760            K       11382, 'Keñeba':      FulaDialects lemma_index
270        Keñeba        1159
696        Keñeba        3030
4651       Keñeba       

[1308, 2760, 4449, 6093, 7911]

In [ ]:
dialects_to_drop = {
   '?':'not the only dialect mentioned: [baabaho]:C<G,N>,Z<?,N>',
}
# notes:
   #F:   (M.S. Diallo, 1990)<F>... Both instances are cited by this author.
      #specific
   #Gimmballa:    # 'beltol ?  n.  (Imperato, 1973)  <Gimmballa>  [pooɗol\xa0?]  cf.:[ɓelt-]'
      #specific
   #Gimmballa(Ñafunke): #  para_text='miɗo  prn,sbj,lf  DFZH  [miɗe]:ZH  [miɗa, meɗe]:H<Soosooɓe(Guumel)>  [ɓiɗo]:H<Gimmballa(Ñafunke)>'
         #para_text='miɗen  prn,sbj,lf  DFH  [ɓiɗe]:H<Gimmballa(Ñafunke)>'
      #just specified by Z and H
   #K    para_text='fotiire (nde) / potiije (ɗe)  DFCZ  CZ<M,K,NE>  [fayannde]'
      #just specified used by C and Z
   #  Keñeba   para_text='bareeru (ndu) / bareeji (ɗi)  DzhYo  Z<FJ>,H<Keñeba>,O<FJ>  [rawaandu]'
      # just specified by Z
   # Motti     para_text='seydi  H  <Motti>'
      # just specified by H
   # S   para_text='faa, fa  DFCZH  C<+>,Z<M,S>  [haa]:CZ<FJ,FT,NE,G>,H<FN>'
      #just specified used by C and Z
   #Soosooɓe(Guumel)    same as : Gimmballa(Ñafunke):       #  para_text='miɗo  prn,sbj,lf  DFZH  [miɗe]:ZH  [miɗa, meɗe]:H<Soosooɓe(Guumel)>  [ɓiɗo]:H<Gimmballa(Ñafunke)>'

   

## saving results to objects

In [ ]:
all_entries[4449].paragraphs

[Docx_Paragraph_and_Runs(para_first_line_indent=-91440, paragraph_enumeration=18410, run_text=['mbabba ', '(mba) / bamɗi (ɗi)  ', 'DCZ  CZ<FJ,FT,M,NE,G,S>  [(pl.) baddi, bamli]:Z  [ndakiire, tefeewa, tefeere]'], para_text='mbabba (mba) / bamɗi (ɗi)  DCZ  CZ<FJ,FT,M,NE,G,S>  [(pl.) baddi, bamli]:Z  [ndakiire, tefeewa, tefeere]', run_font_size_pt=[None, None, 8.0], run_bold=[True, None, None], para_left_indent=46826035380000, run_italic=[None, True, None], run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt']),
 Docx_Paragraph_and_Runs(para_first_line_indent=-91440, paragraph_enumeration=18411, run_text=['donkey'], para_text='donkey', run_font_size_pt=[None], run_bold=[None], para_left_indent=70239053070000, run_italic=[None], run_font_name=['TmsRmn 10pt']),
 Docx_Paragraph_and_Runs(para_first_line_indent=-91440, paragraph_enumeration=18412, run_text=['âne, ânesse'], para_text='âne, ânesse', run_font_size_pt=[None], run_bold=[None], para_left_indent=70239053070000, run_italic=[None], 

In [ ]:
len(all_entries)

7985

In [ ]:
dialected_df = df.copy()
dialected_df['FulaDialects'] = dialected_df['FulaDialects'].apply(lambda x: ','.join(x)).apply(lambda x: x.split(','))

temp = pd.Series(data = dialected_df['FulaDialects'].values, index = dialected_df['lemma_index'])
# print(temp.index.drop_duplicates())
# temp[temp.index.()]
print(temp)

lemma_index
5           [-]
8        [Mali]
11          [-]
14       [Mali]
17       [Mali]
          ...  
32488    [Mali]
32492    [Mali]
32496    [Mali]
32500    [Mali]
32504    [Mali]
Length: 7985, dtype: object


In [79]:
with open('pickled_results/all_dictionary_entries_list.pkl', 'rb') as file:
   all_entries = pickle.load(file, encoding='utf-8')
print(type(all_entries[0]))
for i,dialects in pd.Series(data = dialected_df['FulaDialects'].values, index = dialected_df['lemma_index']).items():
   try:
      all_entries[i].FulaDialects = dialects
   except:
      print(dialects)
      print(i)
      print(all_entries[i])
   # print(all_entries[i])
   # print(e[1])
   # break

<class 'pydantic_fula_dictionary_entries.dict_entry'>
['Mali']
7989


IndexError: list index out of range

In [18]:
pd.reset_option("^display")